In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2 as cv

In [2]:
def find_cameras(ctx):
    devices = ctx.devices

    data = {}

    for dev in devices:
        name = dev.get_info(rs.camera_info.name)
        serial_number = dev.get_info(rs.camera_info.serial_number)

        data[name] = serial_number
        print(
            f"found camera with name: {name} and serial number: {serial_number}"
        )

    return data

x = rs.context()
y = find_cameras(x)
print(y)


found camera with name: Intel RealSense D455 and serial number: 046122251324
{'Intel RealSense D455': '046122251324'}


In [3]:
# Create a context object. This object owns the handles to all connected realsense devices
pipeline_1 = rs.pipeline()
# pipeline_2 = rs.pipeline()

# Configure streams Cam 1
config_1 = rs.config()
config_1.enable_device('046122251324')
config_1.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config_1.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming Cam 1
pipeline_1.start(config_1)
pc_1 = rs.pointcloud()

# Configure streams Cam 2
# config_2 = rs.config()
# config_2.enable_device('046322250790')
# config_2.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
# config_2.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# # Start streaming Cam 2
# pipeline_2.start(config_2)
# pc_2 = rs.pointcloud()
# i = 0

align_to_1 = rs.stream.color
align_1 = rs.align(align_to_1)

# align_to_2 = rs.stream.color
# align_2 = rs.align(align_to_2)
try:
    
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames_1 = pipeline_1.wait_for_frames()
        # Align the depth frame to color frame
        aligned_frames_1 = align_1.process(frames_1)

        # Get aligned frames
        aligned_depth_frame_1 = aligned_frames_1.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame_1 = aligned_frames_1.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame_1 or not color_frame_1:
            continue
        
        # frames_2 = pipeline_2.wait_for_frames()
        # # Align the depth frame to color frame
        # aligned_frames_2 = align_2.process(frames_2)

        # # Get aligned frames
        # aligned_depth_frame_2 = aligned_frames_2.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        # color_frame_2 = aligned_frames_2.get_color_frame()

        # Validate that both frames are valid
        # if not aligned_depth_frame_2 or not color_frame_2:
        #     continue
        # Convert images to numpy arrays
        depth_image_1 = np.asanyarray(aligned_depth_frame_1.get_data())
        color_image_1 = np.asanyarray(color_frame_1.get_data())
        # depth_image_2 = np.asanyarray(aligned_depth_frame_2.get_data())
        # color_image_2 = np.asanyarray(color_frame_2.get_data())
        
        images_1 = color_image_1
        img_1 = depth_image_1
        # images_2 = color_image_2
        # img_2 = depth_image_2
        # Show images
        # cv.namedWindow('RealSense', cv.WINDOW_AUTOSIZE)
        cv.imshow('RealSense1', images_1)
        # cv.imshow('RealSense2', images_2)
        key = cv.waitKey(1)
            
        #PointCloud
        points_1 = pc_1.calculate(aligned_depth_frame_1)
        pc_1.map_to(aligned_depth_frame_1)
        ply_1 = rs.save_to_ply('ply_file_right/depth{}.ply'.format(i))
        ply_1.set_option(rs.save_to_ply.option_ply_binary, False)
        ply_1.set_option(rs.save_to_ply.option_ply_normals, True)
        
        # points_2 = pc_2.calculate(aligned_depth_frame_2)
        # pc_2.map_to(aligned_depth_frame_2)
        # ply_2 = rs.save_to_ply('ply_file_left/depth{}.ply'.format(i))
        # ply_2.set_option(rs.save_to_ply.option_ply_binary, False)
        # ply_2.set_option(rs.save_to_ply.option_ply_normals, True)
                             
        # Capture image
        if key & 0xFF == ord('c'):
            i = i+1
            # Capture video
            # colorwriter_1.write(color_image_1)
            # colorwriter_2.write(color_image_2)
            
            cv.imwrite('image_right/out{}.png'.format(i), images_1)
            cv.imwrite('image_right/img_right/depth{}.png'.format(i), img_1)
            ply_1.process(aligned_depth_frame_1)
            
            # cv.imwrite('image_left/out{}.png'.format(i), images_2)
            # cv.imwrite('image_left/img_left/depth{}.png'.format(i), img_2)
            # ply_2.process(aligned_depth_frame_2)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv.destroyAllWindows()
            break
        
finally:
    # Stop streaming
    pipeline_1.stop()
    # pipeline_2.stop()


NameError: name 'i' is not defined

: 

In [6]:
# Create a context object. This object owns the handles to all connected realsense devices
pipeline_1 = rs.pipeline()
pipeline_2 = rs.pipeline()

# Configure streams Cam 1
config_1 = rs.config()
config_1.enable_device('046122251324')
config_1.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config_1.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

color_path_1 = 'V00P00A00C00_rgb_1.avi'
colorwriter_1 = cv.VideoWriter(color_path_1, cv.VideoWriter_fourcc(*'XVID'), 30, (640,480), 1)
# Start streaming Cam 1
pipeline_1.start(config_1)

# Configure streams Cam 2
config_2 = rs.config()
config_2.enable_device('046322250790')
config_2.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config_2.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

color_path_2 = 'V00P00A00C00_rgb_2.avi'
colorwriter_2 = cv.VideoWriter(color_path_2, cv.VideoWriter_fourcc(*'XVID'), 30, (640,480), 1)
# Start streaming Cam 2
pipeline_2.start(config_2)

align_to_1 = rs.stream.color
align_1 = rs.align(align_to_1)

align_to_2 = rs.stream.color
align_2 = rs.align(align_to_2)
try:
    
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames_1 = pipeline_1.wait_for_frames()
        # Align the depth frame to color frame
        aligned_frames_1 = align_1.process(frames_1)

        # Get aligned frames
        aligned_depth_frame_1 = aligned_frames_1.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame_1 = aligned_frames_1.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame_1 or not color_frame_1:
            continue
        
        frames_2 = pipeline_2.wait_for_frames()
        # Align the depth frame to color frame
        aligned_frames_2 = align_2.process(frames_2)

        # Get aligned frames
        aligned_depth_frame_2 = aligned_frames_2.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame_2 = aligned_frames_2.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame_2 or not color_frame_2:
            continue
        # Convert images to numpy arrays
        depth_image_1 = np.asanyarray(aligned_depth_frame_1.get_data())
        color_image_1 = np.asanyarray(color_frame_1.get_data())
        depth_image_2 = np.asanyarray(aligned_depth_frame_2.get_data())
        color_image_2 = np.asanyarray(color_frame_2.get_data())
        
        images_1 = color_image_1
        img_1 = depth_image_1
        images_2 = color_image_2
        img_2 = depth_image_2
        # Show images
        # cv.namedWindow('RealSense', cv.WINDOW_AUTOSIZE)
        cv.imshow('RealSense1', images_1)
        cv.imshow('RealSense2', images_2)
        key = cv.waitKey(1)
        # Capture video
        colorwriter_1.write(color_image_1)
        colorwriter_2.write(color_image_2)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv.destroyAllWindows()
            break
        
finally:
    # Stop streaming
    pipeline_1.stop()
    pipeline_2.stop()